### Homework 5: Time Series Prediction with LTI Systems
and Neural Networks


## Question 1:

Download the data cho weather.csv, which contains hourly weather at Charlottesville Albemarle Airport (CHO) since 2019
. 
The columns are: 

tmpf Temperature (F)

dwpf Dew Point (F)

relh Relative Humidity (%)

drct Wind Direction (deg)

sped Wind Speed (MPH)

mslp Sea Level Pressure (mb)

p01i Precipitation (in)

Clean the data by filling in missing values with the previous valid value in that column.
Missing values are marked with an ‘M’. The letter ‘T’ indicates a trace amount of precipitation.
Set these values to 0.

In [3]:
import numpy as np
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

df = pandas.read_csv("cho_weather.csv")
data = df.to_numpy()[:, 2:9]
print(data)

for j in range(data.shape[1]):
    for i in range(data.shape[0]):
        if data[i, j] == "M":
            data[i, j] = data[i - 1, j]
        elif data[i, j] == "T":
            data[i, j] = 0.0
            
data = np.float32(data)

N = data.shape[0]
print(data)

ModuleNotFoundError: No module named 'torch'

### Question 2

Let’s use LTI systems to predict the weather. Take the exponential moving average (EMA)
system that we discussed in class:

$y[n] = (1 − g)x[n] + gy[n − 1]$.

Given a time series x[n] for n = 0, . . . , L − 1, we can use this as a prediction model for the
next timepoint x[n + 1] (that our model has not seen yet) by taking the last output, y[n], as
our prediction for x[n + 1]. Implement this exponential averaging system and test it on the
provided data of hourly temperatures. Do the following:

(a) Make predictions for $x[n]$ (using $y[n − 1])$ for $n = 1, . . . , L − 1$. 
Plot these predictions over a plot of the original data. Do this three times, with gain parameters
$g = 0.0, 0.25, 0.5, 0.75$. What difference do you see with the four different parameters?

In [ ]:
def ema(x, g):
    y = np.zeros_like(x)
    y[0] = x[0]
    for n in range(1, len(x)):
        y[n] = (1 - g) * x[n] + g * y[n-1]
    return y

g_values = [0.0, 0.25, 0.5, 0.75]

plt.figure(figsize=(12, 6))
plt.plot(data[:, 0], label='Original Data')

for g in g_values:
    y_pred = ema(data[:, 0], g)
    plt.plot(np.arange(1, len(y_pred)+1), y_pred, label=f'EMA, g={g}')

plt.legend()
plt.xlabel('Time')
plt.ylabel('Temperature (F)')
plt.title('Temperature Predictions using EMA')
plt.show()



In [ ]:
for g in g_values:
    y_pred = ema(data[:, 0], g)
    plt.figure(figsize=(12, 6))
    plt.plot(data[:, 0], label='Original Data')
    plt.plot(np.arange(1, len(y_pred)+1), y_pred, label=f'EMA, g={g}')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Temperature (F)')
    plt.title(f'Temperature Predictions using EMA, g={g}')
    plt.tight_layout()
    plt.show()

(b) Compute the mean absolute error (MAE) of the four different models (g = 0.0, 0.25, 0.5, 0.75).
The MAE is

$MAE = \frac{1}{L-1} \sum_{i=1}^{L-1} |x[n] - y[n-1]|$

Which choice of g gave the best prediction (lowest MAE)?



In [ ]:
def compute_mae(x, y):
    return np.sum(np.abs(x - y)) / (len(x) - 1)

g_values = [0.0, 0.25, 0.5, 0.75]
maes = []

for g in g_values:
    y_pred = ema(data[:, 0], g)
    mae = compute_mae(data[:, 0], y_pred)
    maes.append(mae)
    print(f"MAE for g = {g}: {mae:.4f}")

best_g = g_values[np.argmin(maes)]
best_gSTORED_2B = best_g
print(f"The best performing model has g = {best_g}")

(c) Because this is hourly data, our model is really only predicting one hour into the future,
which isn’t so hard! Let’s predict further into the future. Repeat your exponential
moving average model with a delay of 24 (instead of 1). Now predict x[n + 24] using
y[n]. Plot the predictions over the original data again for g = 0.0, 0.25, 0.5, 0.75. Report
the MAE for each model.

In [ ]:
#def ema_24hr(x, g):
#    y = np.zeros_like(x)
#    y[24:] = (1 - g) * x[:-24] + g * y[:-24]
#    return y

def ema_24hr(x, g):
    y = np.zeros_like(x)
    y[0] = x[0]
    for n in range(24, len(x)):
        y[n] = (1 - g) * x[n-24] + g * y[n-24]
    return y


g_values = [0.0, 0.25, 0.5, 0.75]
maes = []

for g in g_values:
    y_pred = ema_24hr(data[:, 0], g)
    mae = compute_mae(data[24:, 0], y_pred[24:])
    maes.append(mae)
    print(f"MAE for g = {g} with 24-hour delay: {mae:.4f}")

best_g = g_values[np.argmin(maes)]
best_gSTORED_2C = best_g
print(f"The best performing model has g = {best_g}")

(d) Now try running your EMA system with a delay of 8 hours to predict 8 hours ahead.
You don’t need to plot, just report the MAE for the same four gain values. What do
you notice about how this model performs compared to predicting a full 24 hours ahead?
Give a conjecture for why you think the performance difference is they way it is.

In [ ]:
def ema_8hr(x, g):
    y = np.zeros_like(x)
    y[0] = x[0]
    for n in range(8, len(x)):
        y[n] = (1 - g) * x[n-8] + g * y[n-8]
    return y

# Compute MAE for each EMA model with 8-hour delay
g_values = [0.0, 0.25, 0.5, 0.75]
maes = []

for g in g_values:
    y_pred = ema_8hr(data[:, 0], g)
    mae = compute_mae(data[8:, 0], y_pred[8:])
    maes.append(mae)
    print(f"MAE for g = {g} with 8-hour delay: {mae:.4f}")

best_g = g_values[np.argmin(maes)]
best_gSTORED_2D = best_g
print(f"The best performing model has g = {best_g}")

## NEURAL NETWORKS


At the end of this exercise, you’ll have several trained models: 3 linear and 1 RNN (possibly
one more if you do extra credit problem #8). For each model, save it as a *.pt file using
the provided code. You will turn in these models with your code. At the end of
the assignment report the test accuracy (MAE) for all of the competing models. Write a
paragraph about what you learned in terms of which models perform the best, what are the
challenges in getting them to work, and what are the pros and cons of each model.


### Question 3
Now let’s train neural networks to predict the weather. Throughout this part, we will reserve
the last year of data to use as test data to evaluate the performance of our models. The
last year is 365days × 24hrs/day = 8760 time points. First, take your best-performing EMA
models above and evaluate their performance (MAE) on the test data. (You may want to
write a function that runs a model on the test data and outputs its MAE because you’ll need
to repeat this for several models.)

In [ ]:
# Evaluate the best EMA model on the test set
def evaluate_model(model, test_data):
    if isinstance(model, np.ndarray):
        test_predictions = model[-len(test_data):]
    else:
        test_predictions = model(test_data)
    return compute_mae(test_data, test_predictions)

In [ ]:
# Load the best EMA model from part 2b

best_g = best_gSTORED_2B  # Assuming you stored the best gain parameter in this variable
best_ema_model = ema(train_data, best_g)

best_ema_mae = evaluate_model(best_ema_model, test_data)
print(f"Best EMA model with no delay (g={best_g}) MAE on test set: {best_ema_mae:.4f}")

# Compute EMA models with 8-hour and 24-hour delays
ema_8hr_model = ema_8hr(train_data, best_gSTORED_2D)
ema_24hr_model = ema_24hr(train_data, best_gSTORED_2C)

best_g = best_gSTORED_2D

# Evaluate 8-hour and 24-hour EMA models on the test set
ema_8hr_mae = evaluate_model(ema_8hr_model, test_data[8:])
print(f"EMA 8-hour model (g={best_g}) MAE on test set: {ema_8hr_mae:.4f}")

best_g = best_gSTORED_2C

ema_24hr_mae = evaluate_model(ema_24hr_model, test_data[24:])
print(f"EMA 24-hour model (g={best_g}) MAE on test set: {ema_24hr_mae:.4f}")

## Question 4
 Train a single-layer, linear neural network prediction model (LinearPrediction in the provided code). Use at least one full year of timepoints for training data (make sure your training
data is separate from the test data!). Experiment with the window size (how many time points
to incorporate in the prediction), the learning rate, and the number of epochs to train. You
should be able to choose parameters such that the training runs in a reasonably short amount
of time and the final result performs better than EMA on the test data.


In [ ]:
class LinearPredictor(nn.Module):
    def __init__(self, window_size, y_size = 1):
        super(LinearPredictor, self).__init__()
        self.y_size = y_size
        self.window_size = window_size
        self.linear = nn.Linear(window_size, 1)

    ## Here h is a dummy hidden variable, just to make this call the same as an RNN
    def forward(self, x, h):
        y = self.linear(x.reshape(1, self.window_size))
        return y.reshape(self.y_size), h

def train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size = 1):
    N = x.shape[0]

    for epoch in range(num_epochs):
        total_loss = 0.0
        for i in range(window_size, N):
            model.zero_grad()
            
            h = torch.zeros(1, h_size)
            ## this?
            ## y, h = model(x[(i - window_size - offset):(i - offset)], h)
            y, h = model(x[(i - window_size):i], h)

            loss = criterion(y, targets[i])
            loss.backward()

            optimizer.step()
            
            total_loss += loss.detach()

        # convert to mean loss
        total_loss = total_loss / (N - window_size)
            
        print(epoch, ": Training Loss = ", total_loss.item())

# Pull out the temperature time series
temps = data[:, 0]
test_data = data[-8760:, 0]
train_data = data[:-8760, 0]

In [ ]:
# 1 hour prediction - 11.8321 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.001
num_epochs = 3
window_size = 4
x_size = 3
h_size = 5
y_size = 3
# modified above

# model = RNN(x_size, h_size, y_size)
### OR ###
model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 0
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "LNR_1HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("LNR_1 HR.pt"))

### Question 5
Train two more LinearPrediction models, now with the targets being 8 hours (offset =
7) and 24 hours (offset = 23) in the future. You might start with the same parameters you
used in the previous problem and adjust them only if they don’t work well.



In [ ]:
# 8 hour prediction - 9.1836 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.008
num_epochs = 5
window_size = 4
x_size = 4
h_size = 5
y_size = 4
# modified above

# model = RNN(x_size, h_size, y_size)
### OR ###
model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 7
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "LNR_8HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("LNR_8HR.pt"))

In [ ]:
# 24 hour prediction - 11.0766 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.008
num_epochs = 5
window_size = 4
x_size = 4
h_size = 5
y_size = 4
# modified above

# model = RNN(x_size, h_size, y_size)
### OR ###
model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 23
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "LNR_24HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("LNR_24HR.pt"))

### Question 6

Now train an recurrent neural network (RNN) model to predict the temperature one hour
into the future. Now, you will need to experiment with the hidden variable size, the learning
rate, the window size, and the number of epochs. Start small with the hidden variable size
and window size, and increase them if the computation time is not too long.


In [ ]:
# 1 hour prediction - 11.0766 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.008
num_epochs = 3
window_size = 4
x_size = 1
h_size = 3
y_size = 1
# modified above

model = RNN(x_size, h_size, y_size)
### OR ###
# model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 0 # offset is 0, so predicts 1 hr into future
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "RNN_1HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("RNN_1HR.pt"))

### Question 7

Write a function to use your RNN to predict farther into the future by recursively feeding the
outputs yn as the next inputs xn+1. Use this to predict 8 hours into the future and compare
the performance with the EMA and linear neural network models. Repeat this for 24 hours
into the future.


In [ ]:
# 8 hour prediction - 9.1836 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.008
num_epochs = 3
window_size = 4
x_size = 1
h_size = 3
y_size = 1
# modified above

model = RNN(x_size, h_size, y_size)
### OR ###
# model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 7 # offset is 0, so predicts 1 hr into future
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "RNN_8HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("RNN_8HR.pt"))

In [ ]:
# 24 hour prediction - 11.0766 is the MAE to beat

### SET YOUR PARAMETERS HERE! ####
###
learning_rate = 0.008
num_epochs = 3
window_size = 4
x_size = 1
h_size = 3
y_size = 1
# modified above

model = RNN(x_size, h_size, y_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 23 # offset is 0, so predicts 1 hr into future
#modified start, end, offset
L = end - start
#x = torch.Tensor(temps[start:end]).reshape(L, 1)
#targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)
x = torch.Tensor(train_data[start:end]).reshape(L, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "RNN_24HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("RNN_24HR.pt"))

## Question 8
Train an improved neural network such that you can beat the performance of EMA and the
linear neural network in predicting 24 hours into the future. You might consider training the
RNN to directly predict targets 24 hours ahead, modifying the RNN to be an LSTM, using
the other weather variables (wind, pressure, etc.), or other ideas. This is open ended!


In [ ]:
learning_rate = 0.008
num_epochs = 3
window_size = 4
x_size = 1
h_size = 3
y_size = 1

model = nn.LSTM(x_size, h_size, batch_first=True)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
start = 0
end = 8760
offset = 23 # offset is 0, so predicts 1 hr into future
L = end - start
x = torch.Tensor(train_data[start:end]).reshape(L, 1, 1)
targets = torch.Tensor(train_data[(start + offset):(end + offset)]).reshape(L, 1, 1)

# Train the LSTM model
out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size)

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "LSTM_24HR.pt")

# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("LSTM_24HR.pt"))